In [268]:
import pandas as pd
import os
from droughts_modelling.data import DataFunctions
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from droughts_modelling.window_gen import WindowGenerator
from keras.models import load_model


In [7]:
test_data = pd.read_csv("../raw_data/test_timeseries.csv")

In [9]:
WindowGenerator()

NameError: name 'WindowGenerator' is not defined

In [14]:
dataset = DataFunctions(local=True)

In [353]:
test_data = dataset.light_weekly_aggregate_test()

/home/jackhooper/code/realGhostFoxx/droughts_modelling/droughts_modelling/data.py:193: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week_num'] = pd.to_datetime(df['date']).dt.week


In [231]:
year = 2019
week = 11
fips = 1009

shift = 4
inputs = 7

In [233]:
predict_df

,fips_,year_,week_num_,PRECTOT_mean,PS_mean,QV2M_mean,T2M_mean,T2MDEW_mean,T2MWET_mean,T2M_MAX_mean,...,WS10M_MAX_mean,WS10M_MIN_mean,WS10M_RANGE_mean,WS50M_mean,WS50M_MAX_mean,WS50M_MIN_mean,WS50M_RANGE_mean,lat,long,score_max
423,1009,2019,4,6.355714,99.832857,4.240000,3.550000,0.645714,0.488571,9.514286,...,3.254286,1.315714,1.938571,4.618571,6.511429,2.580000,3.932857,33.980867,-86.567371,0.0
424,1009,2019,5,0.611429,99.775714,4.724286,5.318571,2.031429,1.854286,12.815714,...,2.942857,0.990000,1.952857,3.988571,6.244286,1.872857,4.371429,33.980867,-86.567371,0.0
425,1009,2019,6,0.880000,99.937143,8.404286,11.964286,9.801429,9.668571,16.892857,...,3.328571,1.878571,1.448571,5.132857,6.195714,3.695714,2.501429,33.980867,-86.567371,0.0
426,1009,2019,7,8.612857,99.081429,7.711429,11.038571,9.330000,8.992857,16.274286,...,3.477143,1.541429,1.935714,4.864286,6.890000,2.871429,4.020000,33.980867,-86.567371,0.0
427,1009,2019,8,19.545714,99.582857,8.268571,11.872857,10.305714,10.038571,17.082857,...,3.960000,1.505714,2.454286,5.324286,7.248571,3.128571,4.122857,33.980867,-86.567371,0.0
428,1009,2019,9,7.552857,99.615714,7.567143,11.768571,9.211429,9.052857,17.641429,...,2.655714,0.718571,1.937143,3.547143,5.412857,1.394286,4.015714,33.980867,-86.567371,0.0
429,1009,2019,10,2.287143,99.831429,6.038571,7.947143,4.050000,3.934286,13.598571,...,3.454286,1.465714,1.990000,4.721429,6.854286,3.067143,3.787143,33.980867,-86.567371,0.0
430,1009,2019,11,2.457143,99.800000,7.721429,12.827143,9.322857,9.161429,18.224286,...,3.420000,1.582857,1.837143,5.224286,6.938571,3.367143,3.568571,33.980867,-86.567371,0.0


In [235]:
train_data = dataset.light_weekly_aggregate_train()



 
    # st.write(test_df_robust.tail())

    # test_df = test_df_robust.copy()
    # test_ohe = OneHotEncoder(sparse = False)
    # test_ohe.fit(test_df[['score_max']]) 
    # scoremax_encoded_test = test_ohe.transform(test_df[['score_max']])   
    # test_df["score_max_0"],test_df["score_max_1"],test_df['score_max_2'],test_df['score_max_3'],test_df['score_max_4'],test_df['score_max_5'] = scoremax_encoded_test.T     
    # test_df_robust_ohe = test_df.drop(columns=['score_max'])

    # test_df = test_df_robust_ohe[test_df_robust_ohe['fips_'] == fips]


/home/jackhooper/code/realGhostFoxx/droughts_modelling/droughts_modelling/data.py:93: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week_num'] = pd.to_datetime(df['date']).dt.week


In [236]:
features = train_data.drop(columns=['fips_','year_','week_num_','score_max']).columns


In [245]:
train_df = train_data.copy()
test_df = test_data.copy()
                

In [246]:
for f in features:
     train_median = np.median(train_df[f])
     train_iqr = np.subtract(*np.percentile(train_df[f], [75, 25]))
     train_df[f] = train_df[f].map(lambda x: (x-train_median)/train_iqr)
     test_df[f] = test_df[f].map(lambda x: (x-train_median)/train_iqr)
        
test_df_robust = test_df    

In [247]:
test_df = test_df_robust.copy()
test_ohe = OneHotEncoder(sparse = False)
test_ohe.fit(test_df[['score_max']]) 
scoremax_encoded_test = test_ohe.transform(test_df[['score_max']])   
test_df["score_max_0"],test_df["score_max_1"],test_df['score_max_2'],test_df['score_max_3'],test_df['score_max_4'],test_df['score_max_5'] = scoremax_encoded_test.T     
test_df_robust_ohe = test_df.drop(columns=['score_max'])

In [248]:
test_df_robust_ohe

,fips_,year_,week_num_,PRECTOT_mean,PS_mean,QV2M_mean,T2M_mean,T2MDEW_mean,T2MWET_mean,T2M_MAX_mean,...,WS50M_MIN_mean,WS50M_RANGE_mean,lat,long,score_max_0,score_max_1,score_max_2,score_max_3,score_max_4,score_max_5
0,1001,2019,1,-0.470236,0.460926,-0.262398,-0.357672,-0.218069,-0.226309,-0.384225,...,0.551508,-0.945936,-0.827536,0.241557,1.0,0.0,0.0,0.0,0.0,0.0
1,1001,2019,2,-0.019272,0.593352,-0.169715,-0.356480,-0.136660,-0.149753,-0.362898,...,0.202680,-0.650218,-0.827536,0.241557,1.0,0.0,0.0,0.0,0.0,0.0
2,1001,2019,3,0.733191,0.555163,-0.199593,-0.417398,-0.192422,-0.202129,-0.484005,...,-0.155779,-0.508708,-0.827536,0.241557,1.0,0.0,0.0,0.0,0.0,0.0
3,1001,2019,4,1.757173,0.592291,-0.347967,-0.526284,-0.376187,-0.390440,-0.530890,...,-0.039363,-0.148041,-0.827536,0.241557,1.0,0.0,0.0,0.0,0.0,0.0
4,1001,2019,5,-0.236831,0.573904,-0.258740,-0.450115,-0.260659,-0.274391,-0.398358,...,-0.775544,-0.029753,-0.827536,0.241557,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326335,56043,2020,49,-0.440685,-3.603607,-0.712398,-0.958252,-1.209831,-0.924097,-0.954807,...,-0.144891,-0.685776,0.800557,-1.391075,0.0,0.0,0.0,0.0,1.0,0.0
326336,56043,2020,50,-0.338758,-3.749293,-0.684553,-0.973588,-1.147924,-0.901503,-1.057549,...,-0.567002,-0.024673,0.800557,-1.391075,0.0,0.0,0.0,0.0,1.0,0.0
326337,56043,2020,51,-0.188009,-3.834512,-0.648374,-1.044901,-1.043753,-0.888246,-1.100203,...,0.364322,0.024673,0.800557,-1.391075,0.0,0.0,0.0,0.0,1.0,0.0
326338,56043,2020,52,-0.203426,-3.813296,-0.628862,-1.000767,-1.029883,-0.857063,-1.022766,...,1.324958,0.563861,0.800557,-1.391075,0.0,0.0,0.0,0.0,1.0,0.0


In [337]:
fips_df = test_df_robust_ohe[test_data["fips_"] == fips]
year_df = fips_df[fips_df["year_"] == year]
week_df = year_df[year_df["week_num_"] == week]

index = week_df.index[0]

if inputs > (year-2019)*52+week:
    print(f"Pick fewer inputs or later date")
    predict_df = 0
else:
    predict_df = fips_df.loc[index-inputs+1:index+shift,:]

In [352]:
predict_df

,fips_,year_,week_num_,PRECTOT_mean,PS_mean,QV2M_mean,T2M_mean,T2MDEW_mean,T2MWET_mean,T2M_MAX_mean,...,WS50M_MIN_mean,WS50M_RANGE_mean,lat,long,score_max_0,score_max_1,score_max_2,score_max_3,score_max_4,score_max_5
424,1009,2019,5,-0.316916,0.362093,-0.321545,-0.505666,-0.351238,-0.364578,-0.432464,...,-0.657454,-0.028302,-0.621027,0.246696,1.0,0.0,0.0,0.0,0.0,0.0
425,1009,2019,6,-0.236403,0.402051,0.202033,-0.109312,0.155092,0.146111,-0.190928,...,0.411223,-0.978229,-0.621027,0.246696,1.0,0.0,0.0,0.0,0.0,0.0
426,1009,2019,7,2.081799,0.190240,0.103455,-0.164522,0.124372,0.101951,-0.227573,...,-0.072027,-0.206821,-0.621027,0.246696,1.0,0.0,0.0,0.0,0.0,0.0
427,1009,2019,8,5.359315,0.314356,0.182724,-0.114765,0.187954,0.170292,-0.179672,...,0.078727,-0.154572,-0.621027,0.246696,1.0,0.0,0.0,0.0,0.0,0.0
428,1009,2019,9,1.764026,0.322489,0.082927,-0.120985,0.116645,0.105872,-0.146581,...,-0.938023,-0.208999,-0.621027,0.246696,1.0,0.0,0.0,0.0,0.0,0.0
429,1009,2019,10,0.185439,0.375884,-0.134553,-0.348897,-0.219698,-0.228643,-0.386087,...,0.042714,-0.325109,-0.621027,0.246696,1.0,0.0,0.0,0.0,0.0,0.0
430,1009,2019,11,0.236403,0.368105,0.104878,-0.057851,0.123906,0.112968,-0.112051,...,0.218593,-0.436139,-0.621027,0.246696,1.0,0.0,0.0,0.0,0.0,0.0
431,1009,2019,12,-0.480942,0.383310,-0.207724,-0.233450,-0.157233,-0.166651,-0.154621,...,-0.236181,-0.465167,-0.621027,0.246696,1.0,0.0,0.0,0.0,0.0,0.0
432,1009,2019,13,0.012848,0.349364,0.050000,-0.084604,0.087507,0.078331,-0.089455,...,0.134841,-0.373004,-0.621027,0.246696,1.0,0.0,0.0,0.0,0.0,0.0
433,1009,2019,14,0.744754,0.347242,0.213618,0.035784,0.196798,0.186444,0.045870,...,-0.835008,-0.295356,-0.621027,0.246696,1.0,0.0,0.0,0.0,0.0,0.0


In [339]:
wingen = WindowGenerator(predict_df,input_width=inputs,label_width=1,shift=shift,label_columns=["score_max_0","score_max_1","score_max_2","score_max_3","score_max_4","score_max_5"])
predict_data = wingen.split_window


In [340]:
predict_data

<bound method WindowGenerator.split_window of Total window size: 11
Input indices: [0 1 2 3 4 5 6]
Label indices: [10]
Label column name(s): ['score_max_0', 'score_max_1', 'score_max_2', 'score_max_3', 'score_max_4', 'score_max_5']>

In [341]:
predict_data = wingen.make_dataset()

In [342]:
predict_data

<MapDataset shapes: ((None, 7, 29), (None, 1, 6)), types: (tf.float32, tf.float32)>

In [346]:
numpy_images = []
numpy_labels = []

for images, labels in predict_data.take(9):  # only take first element of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

numpy_images

array([[[ 1.00900000e+03,  2.01900000e+03,  5.00000000e+00,
         -3.16916496e-01,  3.62093359e-01, -3.21544707e-01,
         -5.05665839e-01, -3.51238132e-01, -3.64578485e-01,
         -4.32464451e-01, -5.53006828e-01,  6.59023643e-01,
         -5.24298608e-01, -6.52201653e-01, -6.16544604e-01,
         -6.79564714e-01, -5.01285374e-01, -5.55877268e-01,
         -4.02886987e-01, -6.57453954e-01, -2.83018872e-02,
         -6.21027350e-01,  2.46696368e-01,  1.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 1.00900000e+03,  2.01900000e+03,  6.00000000e+00,
         -2.36402571e-01,  4.02050912e-01,  2.02032521e-01,
         -1.09312430e-01,  1.55092165e-01,  1.46111473e-01,
         -1.90927550e-01, -1.67389959e-02, -5.24038434e-01,
         -1.10053442e-01, -3.73324811e-01, -5.00215411e-01,
          7.25513920e-02, -7.28149116e-01, -9.20671672e-02,
         -4.17537421e-01,  4.11222786e-01, -9.78229344e-

In [347]:
model = load_model('trained_model_2021-09-11 13_01_35.778360.h5')


In [348]:
model.predict(predict_data)

array([[[0.33988315, 0.17092684, 0.17421442, 0.12132808, 0.11224885,
         0.08139866],
        [0.46321768, 0.15720907, 0.15766135, 0.09592124, 0.07760903,
         0.04838169],
        [0.51563555, 0.15540412, 0.1427893 , 0.08605886, 0.06238325,
         0.03772898],
        [0.52644783, 0.1615913 , 0.13572213, 0.08396082, 0.05713237,
         0.03514552],
        [0.5227118 , 0.16935666, 0.1328714 , 0.08435778, 0.05559323,
         0.03510909],
        [0.5160434 , 0.17605396, 0.13173006, 0.08521408, 0.05528856,
         0.03566988],
        [0.5100066 , 0.18117604, 0.13123734, 0.08597473, 0.05535198,
         0.03625335]]], dtype=float32)